In [1]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import torch
import dgl
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from dgl.dataloading import GraphDataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from dgl.nn import GraphConv

In [2]:
class processing(nn.Module):  #使用不同的方式来处理数据，普通lstm使用Linear,这里使用GCN。
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fn = GraphConv(input_size,hidden_size) #使用GCN
    def forward(self, g, feat):   #feat:(nodes_num,feature_len)
        feat = self.fn(g, feat)
        feat = F.elu(feat)     #ELU激活函数，可以选择不使用
        return feat
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.forget_gate = processing(input_size + hidden_size, hidden_size)
        self.input_gate = processing(input_size + hidden_size, hidden_size)
        self.output_gate = processing(input_size + hidden_size, hidden_size)
        self.cell_gate = processing(input_size + hidden_size, hidden_size)
    
    def forward(self,g, feature, hidden):
        h_prev, c_prev = hidden
        combined = torch.cat((feature, h_prev), dim=1)
        
        f_t = torch.sigmoid(self.forget_gate(g,combined))
        i_t = torch.sigmoid(self.input_gate(g,combined))
        o_t = torch.sigmoid(self.output_gate(g,combined))
        c_tilde = torch.tanh(self.cell_gate(g,combined))
        
        c_t = f_t * c_prev + i_t * c_tilde
        h_t = o_t * torch.tanh(c_t)
        
        return h_t, c_t
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm_cells = nn.ModuleList([LSTMCell(input_size, hidden_size) if i == 0 else LSTMCell(hidden_size, hidden_size) for i in range(num_layers)])
    
    def forward(self, g,feature,state):
        batch_size, seq_length, _ = feature.size()
        if not state :                 #初始化h,c状态
            h = [torch.zeros(batch_size, self.hidden_size).to(feature.device) for _ in range(self.num_layers)]
            c = [torch.zeros(batch_size, self.hidden_size).to(feature.device) for _ in range(self.num_layers)]
        else:
            h,c = state
        
        #outputs = []
        for t in range(seq_length):
            input_t = feature[:, t, :]
            for layer in range(self.num_layers):
                h[layer], c[layer] = self.lstm_cells[layer](g,input_t, (h[layer], c[layer]))
                input_t = h[layer]
            #outputs.append(h[-1])  #这里我们只需要使用最后的h作为结果，如果需要用到所有结果再进行处理，则需要这样存储起来
        
        #outputs = torch.stack(outputs, dim=1)
        return h[-1], (h, c)
class GCN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size,num_classes,num_layers):
        super().__init__()
        self.Encoder = LSTM(input_size,hidden_size,num_layers) 
        self.Decoder = nn.Linear(hidden_size,num_classes)
    def forward(self, g, feat,state):   #feat:(nodes_num,feature_len)
        h,state = self.Encoder(g,feat,state)
        h = F.elu(h)     #ELU激活函数，可以选择不使用
        h = self.Decoder(h)
        return h,state

In [3]:
graph_size = 12
input_size = 4
seq_length = 12
hidden_size = 12
num_classes = 1
num_layers = 1
nodes_num = 1440
def generate_data(graph_size,nodes_num,seq_length,input_size,num_classes):
    nodes_X = np.random.rand(graph_size,nodes_num, seq_length, input_size)
    nodes_y = np.random.rand(graph_size,nodes_num, num_classes)
    return nodes_X, nodes_y
nodes_X, nodes_y = generate_data(graph_size,nodes_num,seq_length,input_size,num_classes)
#生成模拟数据
X_train = torch.tensor(nodes_X, dtype=torch.float32)
y_train = torch.tensor(nodes_y, dtype=torch.float32) 
#生成边，这里我随机生成的，可以直接根据自己的需要来生成边
# 设置边的生成概率
edge_prob = 0.05
# 构建基于概率的随机边
src, dst = [], []
for i in range(nodes_num):
    for j in range(nodes_num):
        if np.random.rand() < edge_prob:
            src.append(i)
            dst.append(j)

In [4]:
X_train.shape

torch.Size([12, 1440, 12, 4])

In [5]:
y_train.shape

torch.Size([12, 1440, 1])

In [6]:
graphs_train = []
for X, Y in zip(X_train, y_train):
    g = dgl.graph((src,dst))  # 这里只是示例，您可以根据实际情况构建不同的图
    g.ndata['label'] = Y  # 将标签存储在节点特征字典中
    g.ndata['feature'] = X
    graphs_train.append(g)  #将所有的图存在一个图列表里，然后使用dataloader加载数据
loader_train = GraphDataLoader(graphs_train, batch_size=4, shuffle=False,drop_last=False)

In [7]:
model = GCN_LSTM(input_size,hidden_size,num_classes,2).to(device)
state=[]
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss() # 或者根据你的任务选择其他损失函数
for epoch in range(3):  # 假设训练3个epoch
    model.train()
    for batch in loader_train:
        for S in state:
            for s in S:
                s.detach_()      #这里需要注意，我们在使用LSTM进行计算时，通常需要使用这个来将之前的state使用detach避免计算多余的梯度
        batch = batch.to(device)
        optimizer.zero_grad()
        out,state = model(batch,batch.ndata['feature'],state)
        loss = criterion(out, batch.ndata['label'])
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/3] , train-Loss: {loss.item()}')

Epoch [1/3] , train-Loss: 0.35526806116104126
Epoch [2/3] , train-Loss: 0.33588626980781555
Epoch [3/3] , train-Loss: 0.31624722480773926
